<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/%E6%90%AD%E5%BB%BA50%E6%9C%AC%E7%BB%8F%E5%85%B8%E5%B0%8F%E8%AF%B4%E7%9A%84playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [x] 载入数据
- [x] 生成小说list
- [x] 每本小说生成role list
- [x] 搭建Gradio
- [ ] 生成messages
- [ ] 支持prompt编辑

In [1]:
!pip install -q openai
!pip install -q datasets tiktoken

import os
import httpx

os.environ["OPENAI_API_KEY"]  = "set your key here"
# os.environ["OPENAI_API_BASE"] = "如果中转站则使用base"

import openai
from openai import OpenAI
from openai import AsyncOpenAI

from google.colab import userdata

os.environ["OPENAI_API_KEY"]  = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_BASE"] = userdata.get("OPENAI_API_BASE")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 M

In [2]:
!pip install -q gradio
import gradio as gr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.


In [3]:
from datasets import load_dataset

dataset = load_dataset("silk-road/50-Chinese-Novel-Characters")

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
novel_list = []

novel2roles = {}

role2datas = {}

from tqdm import tqdm
for data in tqdm(dataset['train']):
    novel = data['book']
    role = data['role']
    if novel not in novel_list:
        novel_list.append(novel)

    if novel not in novel2roles:
        novel2roles[novel] = []

    if role not in novel2roles[novel]:
        novel2roles[novel].append(role)

    role_tuple = (novel, role)

    if role_tuple not in role2datas:
        role2datas[role_tuple] = []

    role2datas[role_tuple].append(data)




100%|██████████| 156363/156363 [00:18<00:00, 8339.42it/s] 


In [5]:
%cd /content
!rm -rf /content/Zero-Haruhi
!git clone https://github.com/LC1332/Zero-Haruhi
%cd /content/Zero-Haruhi

/content
Cloning into 'Zero-Haruhi'...
remote: Enumerating objects: 440, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 440 (delta 22), reused 18 (delta 11), pack-reused 402
Receiving objects: 100% (440/440), 2.97 MiB | 14.55 MiB/s, done.
Resolving deltas: 100% (294/294), done.
/content/Zero-Haruhi


In [6]:
from ChatHaruhi.utils import base64_to_float_array

from tqdm import tqdm

for novel in tqdm(novel_list):
    for role in novel2roles[novel]:
        for data in role2datas[(novel, role)]:
            data["vec"] = base64_to_float_array(data["bge_zh_s15"])

100%|██████████| 51/51 [00:55<00:00,  1.10s/it]


In [7]:
from ChatHaruhi import ChatHaruhi
from ChatHaruhi.response_openai import get_response


In [8]:
narrators = ["叙述者", "旁白","文章作者","作者","Narrator","narrator"]


def package_persona( role_name, world_name ):
    if role_name in narrators:
        return package_persona_for_narrator( role_name, world_name )

    return f"""I want you to act like {role_name} from {world_name}.
If others‘ questions are related with the novel, please try to reuse the original lines from the novel.
I want you to respond and answer like {role_name} using the tone, manner and vocabulary {role_name} would use."""

def package_persona_for_narrator( role_name, world_name ):
    return f"""I want you to act like narrator {role_name} from {world_name}.
当角色行动之后，继续交代和推进新的剧情."""

In [20]:
# target_book = "凡人修仙传"
# target_role = "韩立"

# if target_book not in novel_list:
#     print(f"novel {target_book} not in dataset")

# if target_role not in novel2roles[target_book]:
#     print(f"role {target_role} not in novel {target_book}")

# stories = [data["story"] for data in role2datas[(target_book, target_role)] ]
# vecs = [data["vec"] for data in role2datas[(target_book, target_role)] ]
# persona = package_persona( target_role, target_book )

# persona += "\n{{RAG对话}}\n{{RAG对话}}\n{{RAG对话}}\n"

In [22]:
# chatbot = ChatHaruhi( role_name = target_role, persona = persona , stories = stories, story_vecs= vecs,\
#                      llm = get_response)

# response = chatbot.chat(user = "", text = "鲁鲁道长: 韩道友近来可好？")
# print(response)

query_rags: [{'n': 1, 'max_token': -1, 'query': '鲁鲁道长: 韩道友近来可好？', 'lid': 3}, {'n': 1, 'max_token': -1, 'query': '鲁鲁道长: 韩道友近来可好？', 'lid': 4}, {'n': 1, 'max_token': -1, 'query': '鲁鲁道长: 韩道友近来可好？', 'lid': 5}] rest_limit = 1697
韩立: 多谢鲁道长关心，我近来一切都好。修炼有所进展，也有一些小事情需要处理，但总体来说还算顺利。鲁道长有什么需要帮忙的吗？


In [23]:
# response = chatbot.chat(user = "", text = "鲁鲁道长: *从纳戒中取出一副地图* 听说道友在找金雷竹，在下刚好获得了一些消息")
# print(response)

query_rags: [{'n': 1, 'max_token': -1, 'query': '鲁鲁道长: *从纳戒中取出一副地图* 听说道友在找金雷竹，在下刚好获得了一些消息', 'lid': 3}, {'n': 1, 'max_token': -1, 'query': '鲁鲁道长: *从纳戒中取出一副地图* 听说道友在找金雷竹，在下刚好获得了一些消息', 'lid': 4}, {'n': 1, 'max_token': -1, 'query': '鲁鲁道长: *从纳戒中取出一副地图* 听说道友在找金雷竹，在下刚好获得了一些消息', 'lid': 5}] rest_limit = 1669
韩立: *眼神一亮* 哦？请鲁道长告诉我，有关金雷竹的消息是什么？


In [16]:
role_tuple2chatbot = {}


def initialize_chatbot( novel, role ):
    global role_tuple2chatbot
    if (novel, role) not in role_tuple2chatbot:
        persona = package_persona( role, novel )
        persona += "\n{{RAG对话}}\n{{RAG对话}}\n{{RAG对话}}\n"
        stories = [data["story"] for data in role2datas[(novel, role)] ]
        vecs = [data["vec"] for data in role2datas[(novel, role)] ]
        chatbot = ChatHaruhi( role_name = role, persona = persona , stories = stories, story_vecs= vecs,\
                             llm = get_response)
        chatbot.verbose = False

        role_tuple2chatbot[(novel, role)] = chatbot

from tqdm import tqdm
for novel in tqdm(novel_list):
    for role in novel2roles[novel]:
        initialize_chatbot( novel, role )

100%|██████████| 51/51 [00:16<00:00,  3.15it/s]


In [27]:
readme_text = """# 使用说明

选择小说角色

如果你有什么附加信息，添加到附加信息里面就可以

比如"韩立会炫耀自己刚刚学会了Python"

然后就可以开始聊天了

因为这些角色还没有增加Greeting信息，所以之后再开发个随机乱聊功能

# 开发细节

- 采用ChatHaruhi3.0的接口进行prompting
- 这里的数据是用一个7B的tuned qwen模型进行抽取的
- 想看数据可以去看第三个tab
- 抽取模型用了40k左右的GLM蒸馏数据
- 抽取模型是腾讯大哥BPSK训练的

# 总结人物性格

第三个Tab里面，可以显示一个prompt总结人物的性格

复制到openai或者GLM或者Claude进行人物总结


# 这些小说数据从HaruhiZero 0.4模型开始，被加入训练

不过当前demo是openai的

"""

In [ ]:
from ChatHaruhi.response_openai import get_response, async_get_response
import gradio as gr

def get_role_list( novel ):
    new_list = novel2roles[novel]
    new_value = new_list[0]
    return gr.update(choices = new_list, value = new_value)

save_log = "/content/output.txt"

def get_chatbot( novel, role ):
    if (novel, role) not in role_tuple2chatbot:
        initialize_chatbot( novel, role )

    return role_tuple2chatbot[(novel, role)]

import json

def submit_chat( novel, role, user_name, user_text, chat_history, persona_addition_info ):

    if len(user_text) > 400:
        user_text = user_text[:400]

    if_user_in_text = True

    chatbot = get_chatbot( novel, role )
    chatbot.persona = initialize_persona( novel, role,  persona_addition_info)
    chatbot.llm_async = async_get_response

    history = []

    for chat_tuple in chat_history:
        if chat_tuple[0] is not None:
            history.append( {"speaker":"{{user}}","content":chat_tuple[0]} )
        if chat_tuple[1] is not None:
            history.append( {"speaker":"{{role}}","content":chat_tuple[1]} )

    chatbot.history = history

    input_text = user_text

    if if_user_in_text:
        input_text = user_name + " : " + user_text
        response = chatbot.chat(user = "", text = input_text )
        # response = await chatbot.async_chat(user = "", text = input_text )
    else:
        response = chatbot.chat(user = user_name, text = input_text)
        # response = await chatbot.async_chat(user = user_name, text = input_text)

    chat_history.append( (input_text, response) )

    print_data = {"novel":novel, "role":role, "user_text":input_text, "response":response}

    print(json.dumps(print_data, ensure_ascii=False))

    with open(save_log, "a",encoding = "utf-8") as f:
        f.write(json.dumps(print_data, ensure_ascii=False) + "\n")

    return chat_history


def initialize_persona( novel, role,  persona_addition_info):
    whole_persona = package_persona( role, novel )
    whole_persona += "\n" + persona_addition_info
    whole_persona += "\n{{RAG对话}}\n{{RAG对话}}\n{{RAG对话}}\n"

    return whole_persona

def clean_history( ):
    return []

def clean_input():
    return ""

import random

def generate_summarize_prompt( novel, role_name ):
    whole_prompt = f'''
你在分析小说{novel}中的角色{role_name}
结合小说{novel}中的内容，以及下文中角色{role_name}的对话
判断{role_name}的人物设定、人物特点以及语言风格

{role_name}的对话:
'''
    stories = [data["story"] for data in role2datas[(novel, role_name)] ]

    sample_n = 5

    sample_stories = random.sample(stories, sample_n)

    for story in sample_stories:
        whole_prompt += story + "\n\n"

    return whole_prompt.strip()


with gr.Blocks() as demo:
    gr.Markdown("""# 50本小说的人物测试

    这个interface由李鲁鲁实现，现在是用openai的，回头替换成HaruhiZero模型

    本来还想实现个随机聊天的功能的，还没来得及写""")

    with gr.Tab("聊天"):
        with gr.Row():
            novel_sel = gr.Dropdown( novel_list, label = "小说", value = "凡人修仙传" , interactive = True)
            role_sel = gr.Dropdown( novel2roles[novel_sel.value], label = "角色", value = "韩立", interactive = True )

        with gr.Row():
            chat_history = gr.Chatbot(height = 600)

        with gr.Row():
            user_name = gr.Textbox(label="user_name", scale = 1, value = "鲁鲁", interactive = True)
            user_text = gr.Textbox(label="user_text", scale = 20)
            submit = gr.Button("submit", scale = 1)

        with gr.Row():
            random_chat = gr.Button("随机聊天", scale = 1)
            clean_message = gr.Button("清空聊天", scale = 1)

        with gr.Row():
            persona_addition_info = gr.TextArea( label = "额外人物设定", value = "",  interactive = True  )

        with gr.Row():
            update_persona = gr.Button("补充人物设定到prompt", scale = 1)

        with gr.Row():
            whole_persona = gr.TextArea( label = "完整的system prompt", value = "",  interactive = False  )

        novel_sel.change(fn = get_role_list, inputs = [novel_sel], outputs = [role_sel]).then(fn = initialize_persona, inputs = [novel_sel, role_sel, persona_addition_info], outputs = [whole_persona])

        role_sel.change(fn = initialize_persona, inputs = [novel_sel, role_sel, persona_addition_info], outputs = [whole_persona])

        update_persona.click(fn = initialize_persona, inputs = [novel_sel, role_sel, persona_addition_info], outputs = [whole_persona])


        user_text.submit(fn = submit_chat, inputs = [novel_sel, role_sel, user_name, user_text, chat_history, persona_addition_info], outputs = [chat_history]).then(fn = clean_input, inputs = [], outputs = [user_text])
        submit.click(fn = submit_chat, inputs = [novel_sel, role_sel, user_name, user_text, chat_history, persona_addition_info], outputs = [chat_history]).then(fn = clean_input, inputs = [], outputs = [user_text])

        clean_message.click(fn = clean_history, inputs = [], outputs = [chat_history])

    with gr.Tab("README"):
        gr.Markdown(readme_text)

    with gr.Tab("辅助人物总结"):
        with gr.Row():
            generate_prompt = gr.Button("生成人物总结prompt", scale = 1)

        with gr.Row():
            whole_prompt = gr.TextArea( label = "复制这个prompt到Openai或者GLM或者Claude进行总结", value = "",  interactive = False  )

        generate_prompt.click(fn = generate_summarize_prompt, inputs = [novel_sel, role_sel], outputs = [whole_prompt])





demo.launch(share=True, debug = True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://2d08378b4fd94175bc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


{"novel": "凡人修仙传", "role": "韩立", "user_text": "鲁鲁 : 我要怎样才能化神啊？", "response": "韩立: 化神这一步可不是那么容易的。首先，你需要达到元婴后期的修为境界，这是化神的基础。然后，你需要找到一种适合自己的化神法门，这通常需要一位高级修士的指点或者是机缘巧合。接着，你需要准备大量的灵力和灵草灵物，用来支撑化神过程中的能量消耗。最后，你需要在一个适合的时间和地点，进行一次极为危险的化神仪式。这个过程中，你将面临巨大的灵力冲击和灵魂的洗礼。只有成功度过这一关，你才能真正化神。但要记住，化神是一条充满风险和挑战的道路，需要付出巨大的努力和牺牲。"}
{"novel": "凡人修仙传", "role": "韩立", "user_text": "鲁鲁 : 一起吃饺子吧", "response": "韩立: 饺子？哈哈，好啊！我很久没吃过饺子了。不过，你得先告诉我，这里哪里有好吃的饺子吗？"}


In [ ]:
#备份


def submit_chat( novel, role, user_name, user_text, chat_history, persona_addition_info ):

    if len(user_text) > 400:
        user_text = user_text[:400]

    if_user_in_text = True

    chatbot = get_chatbot( novel, role )
    chatbot.persona = initialize_persona( novel, role,  persona_addition_info)

    history = []

    for chat_tuple in chat_history:
        if chat_tuple[0] is not None:
            history.append( {"speaker":"{{user}}","content":chat_tuple[0]} )
        if chat_tuple[1] is not None:
            history.append( {"speaker":"{{role}}","content":chat_tuple[1]} )

    chatbot.history = history

    input_text = user_text

    if if_user_in_text:
        input_text = user_name + " : " + user_text
        response = chatbot.chat(user = "", text = input_text )
    else:
        response = chatbot.chat(user = user_name, text = input_text)

    chat_history.append( (user_text, response) )

    print_data = {"novel":novel, "role":role, "user_text":input_text, "response":response}

    print(json.dumps(print_data, ensure_ascii=False))

    with open(save_log, "a",encoding = "utf-8") as f:
        f.write(json.dumps(print_data, ensure_ascii=False) + "\n")

    return chat_history